In [0]:
import pandas as pd
import numpy as np
from fbprophet import Prophet 

## CA WK1
# train_file = 'https://raw.githubusercontent.com/danielcaraway/data/master/covid19_0324/ca_train.csv'
# test_file = 'https://raw.githubusercontent.com/danielcaraway/data/master/covid19_0324/ca_test.csv'
# sub_file = 'https://raw.githubusercontent.com/danielcaraway/data/master/covid19_0324/ca_submission.csv'

## GLOBAL WK 1
# train_file = "https://raw.githubusercontent.com/danielcaraway/data/master/covid19-global-forecasting-week-1/train.csv"
# test_file = "https://raw.githubusercontent.com/danielcaraway/data/master/covid19-global-forecasting-week-1/test.csv"
# sub_file = "https://raw.githubusercontent.com/danielcaraway/data/master/covid19-global-forecasting-week-1/submission.csv"

## GLOBAL WK 2
train_file = "https://raw.githubusercontent.com/danielcaraway/data/master/covid19-global-forecasting-week-2/train.csv"
test_file = "https://raw.githubusercontent.com/danielcaraway/data/master/covid19-global-forecasting-week-2/test.csv"
sub_file = "https://raw.githubusercontent.com/danielcaraway/data/master/covid19-global-forecasting-week-2/submission.csv"

train = pd.read_csv(train_file)
test = pd.read_csv(test_file)
sub = pd.read_csv(sub_file)

In [2]:
train

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
0,1,NaN,Afghanistan,2020-01-22,0.0,0.0
1,2,NaN,Afghanistan,2020-01-23,0.0,0.0
2,3,NaN,Afghanistan,2020-01-24,0.0,0.0
3,4,NaN,Afghanistan,2020-01-25,0.0,0.0
4,5,NaN,Afghanistan,2020-01-26,0.0,0.0
...,...,...,...,...,...,...
19399,29362,NaN,Zimbabwe,2020-03-23,3.0,1.0
19400,29363,NaN,Zimbabwe,2020-03-24,3.0,1.0
19401,29364,NaN,Zimbabwe,2020-03-25,3.0,1.0
19402,29365,NaN,Zimbabwe,2020-03-26,3.0,1.0


## SIDE QUEST: Data prep for chart races

In [0]:
# from google.colab import files
# train_file = "https://raw.githubusercontent.com/danielcaraway/data/master/covid19-global-forecasting-week-2/train.csv"
# train_group = pd.DataFrame(train.groupby(['Country_Region','Date'])['ConfirmedCases'].sum())
# train_group.reset_index(inplace = True)
# train_group.columns = ['name', 'date', 'value']
# train_group.to_csv('covid_wk2_grouped_confirmed.csv', index=False)
# files.download("covid_wk2_grouped_confirmed.csv")
# train_group = pd.DataFrame(train.groupby(['Country_Region','Date'])['Fatalities'].sum())
# train_group.reset_index(inplace = True)
# train_group = train_group[train_group['Fatalities'] > 0]
# train_group.columns = ['name', 'date', 'value']
# train_group.to_csv('covid_wk2_grouped_fatalities.csv', index=False)
# files.download("covid_wk2_grouped_fatalities.csv")

# KAGGLE

In [0]:
  df_train = train.copy()
  df_test = test.copy()

In [0]:
df_train.rename(columns={'Country_Region':'Country'}, inplace=True)
df_test.rename(columns={'Country_Region':'Country'}, inplace=True)

df_train.rename(columns={'Province_State':'State'}, inplace=True)
df_test.rename(columns={'Province_State':'State'}, inplace=True)

In [0]:
df_train['Date'] = pd.to_datetime(df_train['Date'], infer_datetime_format=True)
df_test['Date'] = pd.to_datetime(df_test['Date'], infer_datetime_format=True)

In [7]:
y1_Train = df_train.iloc[:, -2]
y1_Train.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: ConfirmedCases, dtype: float64

In [8]:
y2_Train = df_train.iloc[:, -1]
y2_Train.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: Fatalities, dtype: float64

In [0]:
EMPTY_VAL = "EMPTY_VAL"

def fillState(state, country):
    if state == EMPTY_VAL: return country
    return state

In [10]:
#X_Train = df_train.loc[:, ['State', 'Country', 'Date']]
X_Train = df_train.copy()

X_Train['State'].fillna(EMPTY_VAL, inplace=True)
X_Train['State'] = X_Train.loc[:, ['State', 'Country']].apply(lambda x : fillState(x['State'], x['Country']), axis=1)

X_Train.loc[:, 'Date'] = X_Train.Date.dt.strftime("%m%d")
X_Train["Date"]  = X_Train["Date"].astype(int)

X_Train.head()

,Id,State,Country,Date,ConfirmedCases,Fatalities
0,1,Afghanistan,Afghanistan,122,0.0,0.0
1,2,Afghanistan,Afghanistan,123,0.0,0.0
2,3,Afghanistan,Afghanistan,124,0.0,0.0
3,4,Afghanistan,Afghanistan,125,0.0,0.0
4,5,Afghanistan,Afghanistan,126,0.0,0.0


In [11]:
#X_Test = df_test.loc[:, ['State', 'Country', 'Date']]
X_Test = df_test.copy()

X_Test['State'].fillna(EMPTY_VAL, inplace=True)
X_Test['State'] = X_Test.loc[:, ['State', 'Country']].apply(lambda x : fillState(x['State'], x['Country']), axis=1)

X_Test.loc[:, 'Date'] = X_Test.Date.dt.strftime("%m%d")
X_Test["Date"]  = X_Test["Date"].astype(int)

X_Test.head()

,ForecastId,State,Country,Date
0,1,Afghanistan,Afghanistan,319
1,2,Afghanistan,Afghanistan,320
2,3,Afghanistan,Afghanistan,321
3,4,Afghanistan,Afghanistan,322
4,5,Afghanistan,Afghanistan,323


In [12]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

X_Train.Country = le.fit_transform(X_Train.Country)
X_Train['State'] = le.fit_transform(X_Train['State'])

X_Train.head()

,Id,State,Country,Date,ConfirmedCases,Fatalities
0,1,0,0,122,0.0,0.0
1,2,0,0,123,0.0,0.0
2,3,0,0,124,0.0,0.0
3,4,0,0,125,0.0,0.0
4,5,0,0,126,0.0,0.0


In [13]:
X_Test.Country = le.fit_transform(X_Test.Country)
X_Test['State'] = le.fit_transform(X_Test['State'])

X_Test.head()

,ForecastId,State,Country,Date
0,1,0,0,319
1,2,0,0,320
2,3,0,0,321
3,4,0,0,322
4,5,0,0,323


In [0]:
from sklearn.model_selection import GridSearchCV 
import time 
param_grid = {'n_estimators': [1000]}

def gridSearchCV(model, X_Train, y_Train, param_grid, cv=10, scoring='neg_mean_squared_error'): 
  start = time.time()

In [0]:
# grid_cv = GridSearchCV(model, param_grid, cv=10, scoring="neg_mean_squared_error")
# grid_cv.fit(X_Train, y_Train)

#print (f'{type(model).__name__} Hyper Paramter Tuning took a Time: {time.time() - start}')
#print (f'Best {scoring}: {grid_cv.best_score_}')
#print ("Best Hyper Parameters:\n{}".format(grid_cv.best_params_))

# return grid_cv.best_estimator_

In [0]:
from xgboost import XGBRegressor

model = XGBRegressor()

model1 = gridSearchCV(model, X_Train, y1_Train, param_grid, 10, 'neg_mean_squared_error') 
model2 = gridSearchCV(model, X_Train, y2_Train, param_grid, 10, 'neg_mean_squared_error')

In [0]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

In [19]:
from xgboost import XGBRegressor

countries = X_Train.Country.unique()

#models_C = {}
#models_F = {}

df_out = pd.DataFrame({'ForecastId': [], 'ConfirmedCases': [], 'Fatalities': []})

for country in countries:
    states = X_Train.loc[X_Train.Country == country, :].State.unique()
    #print(country, states)
    # check whether string is nan or not
    for state in states:
        X_Train_CS = X_Train.loc[(X_Train.Country == country) & (X_Train.State == state), ['State', 'Country', 'Date', 'ConfirmedCases', 'Fatalities']]
        
        y1_Train_CS = X_Train_CS.loc[:, 'ConfirmedCases']
        y2_Train_CS = X_Train_CS.loc[:, 'Fatalities']
        
        X_Train_CS = X_Train_CS.loc[:, ['State', 'Country', 'Date']]
        
        X_Train_CS.Country = le.fit_transform(X_Train_CS.Country)
        X_Train_CS['State'] = le.fit_transform(X_Train_CS['State'])
        
        X_Test_CS = X_Test.loc[(X_Test.Country == country) & (X_Test.State == state), ['State', 'Country', 'Date', 'ForecastId']]
        
        X_Test_CS_Id = X_Test_CS.loc[:, 'ForecastId']
        X_Test_CS = X_Test_CS.loc[:, ['State', 'Country', 'Date']]
        
        X_Test_CS.Country = le.fit_transform(X_Test_CS.Country)
        X_Test_CS['State'] = le.fit_transform(X_Test_CS['State'])
        
        #models_C[country] = gridSearchCV(model, X_Train_CS, y1_Train_CS, param_grid, 10, 'neg_mean_squared_error')
        #models_F[country] = gridSearchCV(model, X_Train_CS, y2_Train_CS, param_grid, 10, 'neg_mean_squared_error')
        
        model1 = XGBRegressor(n_estimators=1000)
        model1.fit(X_Train_CS, y1_Train_CS)
        y1_pred = model1.predict(X_Test_CS)
        
        model2 = XGBRegressor(n_estimators=1000)
        model2.fit(X_Train_CS, y2_Train_CS)
        y2_pred = model2.predict(X_Test_CS)
        
        df = pd.DataFrame({'ForecastId': X_Test_CS_Id, 'ConfirmedCases': y1_pred, 'Fatalities': y2_pred})
        df_out = pd.concat([df_out, df], axis=0)
    # Done for state loop
# Done for country Loop

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:588: FutureWarning:

Series.base is deprecated and will be removed in a future version



[22:04:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:04:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:04:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:04:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:04:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:04:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:04:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:04:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [20]:
df_out.ForecastId = df_out.ForecastId.astype('int')
df_out.tail()

,ForecastId,ConfirmedCases,Fatalities
12637,12638,4.998635,0.999577
12638,12639,4.998635,0.999577
12639,12640,4.998635,0.999577
12640,12641,4.998635,0.999577
12641,12642,4.998635,0.999577


In [0]:
df_out.to_csv('submission.csv', index=False)